In [60]:
import os, sys, json

parentdir = os.path.abspath('/home/xcheng/mvesc/ETL')
sys.path.insert(0,parentdir)
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Image
from IPython.display import display
from numpy import random
import pickle
from estimate_prediction_model import *
from write_to_database import write_scores_to_db
from optparse import OptionParser
import re
from make_predictions_for_unlabeled_students import * 
import sqlalchemy
%load_ext autotime
%matplotlib inline

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 10.3 ms


In [67]:
json_filename = 'feature_name_mapping_to_human_readable.json'
with postgres_pgconnection_generator() as conn:
    with conn.cursor() as cursor:
        sql_all_table_names = """
        SELECT table_name FROM information_schema.tables 
        WHERE table_schema='model';
        """
        #table_names = pd.read_sql_query(sql_all_table_names, conn)
        # copied feature table names to the list below
        table_names = ['demographics', 'snapshots', 'absence', 'mobility', 'grades', 'oaa_normalized', 'intervention']
        sql_colnames = """
        select table_name, column_name, udt_name from information_schema.columns 
        where table_schema='model' and table_name in ({tables})
        """.format(tables = "\'" + "\', \'".join(table_names) + "\'")
        tables_columns_dtypes = pd.read_sql_query(sql_colnames, conn)
            
colnames_grade = set(filter(lambda x: '_gr_' in x, tables_columns_dtypes.column_name))
colnames_others = set(filter(lambda x: not ('_gr_' in x), tables_columns_dtypes.column_name))
all_tokens = []
for nm in colnames_grade:
    all_tokens = all_tokens + nm.split('_')
all_tokens = set(all_tokens)
token_mapping = {
 'avg': 'average',
 'change':'changes',
 'consec':'consecutive',
 'ed':'education',
 'extracurr': 'extracurricular',
 'instruc':'instruction',
 'intracurr':'intracurricular', 
 'inv':'intervention',
 'iss':'in school suspension',
 'n':'number of', 
 'num':'number of',
 'oss':'out of school suspension',
'percent':'percentage',
 'pf':'pass_or_fail',
 'pl':'performance level',
 'titlei':'titleI',
 'to':'',
 'withdraw':'withdrawal',
 'wkd':'on weekday'}
mapping = {}
# names make sense themselves
for nm in colnames_others:
    mapping[nm] = nm
for nm in colnames_grade:
    tokens = nm.split('_')
    new_nm = ''
    for t in tokens:
        if t=='gr':
            if 'to' in tokens:
                new_nm = new_nm + 'upto grade'
            else:
                new_nm = new_nm + 'in grade '
        elif t in token_mapping:
            new_nm = new_nm + token_mapping[t] + ' '
        else:
            new_nm = new_nm + t + ' '
    mapping[nm] = new_nm

with open(json_filename, 'w') as f:
    json.dump(mapping, f, ensure_ascii=True, sort_keys=True, indent=4)

time: 292 ms


In [59]:
mapping

{'absence_consec_gr_10': 'absence consecutive at grade 10 ',
 'absence_consec_gr_11': 'absence consecutive at grade 11 ',
 'absence_consec_gr_12': 'absence consecutive at grade 12 ',
 'absence_consec_gr_3': 'absence consecutive at grade 3 ',
 'absence_consec_gr_4': 'absence consecutive at grade 4 ',
 'absence_consec_gr_5': 'absence consecutive at grade 5 ',
 'absence_consec_gr_6': 'absence consecutive at grade 6 ',
 'absence_consec_gr_7': 'absence consecutive at grade 7 ',
 'absence_consec_gr_8': 'absence consecutive at grade 8 ',
 'absence_consec_gr_9': 'absence consecutive at grade 9 ',
 'absence_gr_10': 'absence at grade 10 ',
 'absence_gr_11': 'absence at grade 11 ',
 'absence_gr_12': 'absence at grade 12 ',
 'absence_gr_3': 'absence at grade 3 ',
 'absence_gr_4': 'absence at grade 4 ',
 'absence_gr_5': 'absence at grade 5 ',
 'absence_gr_6': 'absence at grade 6 ',
 'absence_gr_7': 'absence at grade 7 ',
 'absence_gr_8': 'absence at grade 8 ',
 'absence_gr_9': 'absence at grade 9 '

time: 27.8 ms


In [46]:
all_tokens

{'1',
 '10',
 '11',
 '12',
 '2',
 '3',
 '4',
 '5',
 '504',
 '6',
 '7',
 '8',
 '9',
 'absence',
 'absent',
 'academic',
 'address',
 'addresses',
 'art',
 'atheletics',
 'avg',
 'change',
 'cities',
 'city',
 'classes',
 'consec',
 'days',
 'disability',
 'disadvantagement',
 'discipline',
 'district',
 'districts',
 'ed',
 'english',
 'excused',
 'extracurr',
 'future',
 'gifted',
 'gpa',
 'gr',
 'health',
 'humanities',
 'in',
 'incidents',
 'instruc',
 'interventions',
 'intracurr',
 'inv',
 'iss',
 'language',
 'limited',
 'math',
 'medical',
 'mid',
 'n',
 'normalized',
 'num',
 'oss',
 'passed',
 'percent',
 'percentile',
 'pf',
 'pl',
 'placement',
 'plan',
 'post',
 'prep',
 'present',
 'program',
 'read',
 'records',
 'school',
 'science',
 'secondary',
 'section',
 'socstudies',
 'spec',
 'special',
 'status',
 'stem',
 'street',
 'tardy',
 'titlei',
 'to',
 'transition',
 'unexcused',
 'vocational',
 'withdraw',
 'wkd',
 'year'}

time: 3.28 ms


In [35]:
{
 'avg': 'average',
 'consec':'consecutive',
 'ed':'education',
 'extracurr': 'extracurricular',
 'gr':'grade',
 'instruc':'instruction'
 'intracurr':'intracurricular', 
 'inv':'intervention',
 'iss':'in school suspension',
 'n',
 'num':'number',
 'oss':'out of school suspension',
 'pf':'pass_or_fail',
 'pl':'performance level',
 'titlei',
 'wkd':'weekday'}

(794, 3)

time: 2.76 ms


In [34]:
len(list(filter(lambda x: (x in colnames_grade) or (x in colnames_others) , tables_columns_dtypes.column_name)))

794

time: 9.53 ms
